In [ ]:
# Export results as JSON (equivalent to the original script's output)
import json

# Pretty print the results
print("Results JSON (equivalent to method_out.json):")
print("=" * 50)
print(json.dumps(results, indent=2))

# Optional: Save to file (uncomment if needed)
# with open("method_out.json", "w") as f:
#     json.dump(results, f, indent=2)
# print("Results saved to method_out.json")

## Results Export

The original script saved results to JSON. Here's the equivalent output:

In [ ]:
def run_experiment_with_params(data, epsilon_target=0.10, delta=0.05, min_samples=100, hysteresis=0.05):
    """Run experiment with custom parameters."""
    np.random.seed(42)  # Keep same seed for fair comparison
    
    controller = DKWController(
        epsilon_target=epsilon_target,
        delta=delta,
        min_samples=min_samples,
        hysteresis=hysteresis
    )
    
    results = {"baseline": [], "proposed": []}
    
    for example in data:
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",
            "error": error,
        })

    return results

# Try different parameter settings
print("Experimenting with different parameters:")
print("=" * 50)

# More aggressive setting (lower epsilon_target)
results_aggressive = run_experiment_with_params(sample_data, epsilon_target=0.05)
aggressive_fusions = sum(1 for r in results_aggressive["proposed"] if r["decision"] == "fusion")
aggressive_errors = sum(1 for r in results_aggressive["proposed"] if r["error"])

print(f"Aggressive (epsilon_target=0.05): {aggressive_fusions} fusions, {aggressive_errors} errors")

# More conservative setting (higher epsilon_target)
results_conservative = run_experiment_with_params(sample_data, epsilon_target=0.20)
conservative_fusions = sum(1 for r in results_conservative["proposed"] if r["decision"] == "fusion")
conservative_errors = sum(1 for r in results_conservative["proposed"] if r["error"])

print(f"Conservative (epsilon_target=0.20): {conservative_fusions} fusions, {conservative_errors} errors")

# Default setting
default_fusions = sum(1 for r in results["proposed"] if r["decision"] == "fusion")
default_errors = sum(1 for r in results["proposed"] if r["error"])

print(f"Default (epsilon_target=0.10): {default_fusions} fusions, {default_errors} errors")

## Interactive Parameter Exploration

Try modifying the DKW controller parameters to see how they affect performance:

In [ ]:
# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# Extract data for plotting
example_ids = [r['id'] for r in results['baseline']]
difficulties = [d['difficulty'] for d in sample_data]
baseline_decisions = [1 if r['decision'] == 'fusion' else 0 for r in results['baseline']]
proposed_decisions = [1 if r['decision'] == 'fusion' else 0 for r in results['proposed']]
errors = [1 if r['error'] else 0 for r in results['baseline']]

# Plot 1: Decision patterns
ax1.bar(range(len(example_ids)), baseline_decisions, alpha=0.7, label='Baseline', color='red')
ax1.bar([x + 0.4 for x in range(len(example_ids))], proposed_decisions, alpha=0.7, label='Proposed', color='blue', width=0.4)
ax1.set_ylabel('Fusion Decision (1=Fusion, 0=Fission)')
ax1.set_xlabel('Example Index')
ax1.set_title('Decision Patterns')
ax1.legend()
ax1.set_xticks(range(len(example_ids)))

# Plot 2: Difficulty vs Errors
ax2.scatter(difficulties, errors, alpha=0.7, s=50)
ax2.set_xlabel('Difficulty Level')
ax2.set_ylabel('Error Occurred (1=Error, 0=Success)')
ax2.set_title('Difficulty vs Error Rate')
ax2.grid(True, alpha=0.3)

# Plot 3: Strategy comparison
strategies = ['Baseline\n(Always Fission)', 'Proposed\n(DKW Controller)']
error_counts = [baseline_errors, proposed_errors]
fusion_counts = [baseline_fusions, proposed_fusions]

x_pos = range(len(strategies))
ax3.bar(x_pos, error_counts, alpha=0.7, color=['red', 'blue'])
ax3.set_ylabel('Number of Errors')
ax3.set_title('Error Comparison')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(strategies)

ax4.bar(x_pos, fusion_counts, alpha=0.7, color=['red', 'blue'])
ax4.set_ylabel('Number of Fusion Decisions')
ax4.set_title('Fusion Decision Comparison')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(strategies)

plt.tight_layout()
plt.show()

## Visualization and Analysis

Let's create some visualizations to better understand the controller's behavior:

In [ ]:
# Run the experiment
results = run_experiment(sample_data)

# Display results
print("Experimental Results:")
print("====================")
print()

print("Baseline (Always Fission):")
for result in results["baseline"]:
    print(f"  {result['id']}: decision={result['decision']}, error={result['error']}")

print()
print("Proposed (DKW Controller):")
for result in results["proposed"]:
    print(f"  {result['id']}: decision={result['decision']}, error={result['error']}")

# Calculate summary statistics
baseline_errors = sum(1 for r in results["baseline"] if r["error"])
proposed_errors = sum(1 for r in results["proposed"] if r["error"])
baseline_fusions = sum(1 for r in results["baseline"] if r["decision"] == "fusion")
proposed_fusions = sum(1 for r in results["proposed"] if r["decision"] == "fusion")

print()
print("Summary:")
print(f"Baseline - Errors: {baseline_errors}/{len(results['baseline'])}, Fusions: {baseline_fusions}")
print(f"Proposed - Errors: {proposed_errors}/{len(results['proposed'])}, Fusions: {proposed_fusions}")

## Running the Experiment

Let's run the experiment and examine the results:

In [ ]:
def run_experiment(data):
    """Run DKW controller experiment with inline data."""
    # Set random seed for reproducible results
    np.random.seed(42)
    
    controller = DKWController()
    results = {"baseline": [], "proposed": []}

    for example in data:
        # Simulate error occurrence based on difficulty
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",  # Always conservative
            "error": error,
        })

    return results

## Experiment Function

The experiment compares two strategies:
1. **Baseline**: Always uses fission (conservative approach)
2. **Proposed**: Uses the DKW controller to adaptively switch between fusion and fission

In [ ]:
# Sample dataset - inline data instead of reading from external JSON
sample_data = [
    {"id": "example_000", "difficulty": 0.1},  # Easy example, low error probability
    {"id": "example_001", "difficulty": 0.3},  # Medium example
    {"id": "example_002", "difficulty": 0.8},  # Hard example, high error probability
    {"id": "example_003", "difficulty": 0.2},  # Easy-medium
    {"id": "example_004", "difficulty": 0.6},  # Medium-hard
    {"id": "example_005", "difficulty": 0.1},  # Easy
    {"id": "example_006", "difficulty": 0.9},  # Very hard
    {"id": "example_007", "difficulty": 0.4},  # Medium
    {"id": "example_008", "difficulty": 0.15}, # Easy
    {"id": "example_009", "difficulty": 0.7},  # Hard
]

print(f"Created sample dataset with {len(sample_data)} examples")
print("Sample entries:")
for i in range(3):
    print(f"  {sample_data[i]}")

## Sample Dataset

Instead of reading from external JSON files, we'll create sample data inline. The dataset contains examples with varying difficulty levels that simulate real-world scenarios.

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10
    delta: float = 0.05
    min_samples: int = 100
    hysteresis: float = 0.05

    samples: list = field(default_factory=list)
    current_state: str = "fission"

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

In [ ]:
"""DKW Controller Implementation."""
import json
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field

# DKW Controller Implementation - experiment_001

This notebook demonstrates a **DKW-guided fusion/fission controller** that uses the Dvoretzky–Kiefer–Wolfowitz (DKW) inequality to make adaptive decisions between fusion and fission strategies based on error observations.

## Overview
The DKW Controller maintains statistical confidence bounds on error rates and switches between:
- **Fusion**: More aggressive strategy when error rates are low
- **Fission**: Conservative strategy when error rates are high

The controller uses hysteresis to prevent rapid switching and provides statistical guarantees via the DKW inequality.